In [1]:
import pandas as pd
import numpy as np

In [2]:
class Location:
    row:int
    col:int
    def __init__(self, row, col) -> None:
        self.row=row
        self.col=col
    def text(self):
        print(self.row, self.col)

In [3]:
def getMinLocation(distance_matrix_l1_init:pd.DataFrame):
    matrix2min = 9999
    matrix2RowColMin=Location(9999,9999)
    matrixsize = distance_matrix_l1_init.shape[0]
    for i in range(matrixsize):
        for j in range(matrixsize):
            if distance_matrix_l1_init.iloc[i,j]<matrix2min and distance_matrix_l1_init.iloc[i,j]>0:
                matrix2min = distance_matrix_l1_init.iloc[i,j]
                matrix2RowColMin = Location(i,j)
    print("Min Location==>",matrix2RowColMin.text, "Min value==>", matrix2min)
    print("Row==>", distance_matrix_l1_init.columns[matrix2RowColMin.row], "Col==>",distance_matrix_l1_init.columns[matrix2RowColMin.col] )
    return matrix2RowColMin

In [4]:
def updateDistances (distanceMatrix:pd.DataFrame, minLocation):
    distance_matrix_l1_init = distanceMatrix.copy()
    colNames = distance_matrix_l1_init.columns
#     lowerIndex, upperIndex
    comboCol1Name = f"S{minLocation[0]+1}"
    comboCol2Name = f"S{minLocation[1]+1}"
    for i in range(1, len(colNames),1):
        val1 = distance_matrix[comboCol1Name][colNames[i]]
        val2 = distance_matrix[colNames[i]][comboCol2Name]    
        distance_matrix_l1_init.iloc[0,i] = (val1 + val2)/2.0
        distance_matrix_l1_init.iloc[i,0] = (val1 + val2)/2.0
        print(f"{comboCol1Name}_{colNames[i]}({val1})".ljust(10, ' '),f"{comboCol2Name}_{colNames[i]}({val2})".ljust(10, ' '))
    return distance_matrix_l1_init

In [5]:
def removeMinDistanceRowCol(distanceMatrix:pd.DataFrame, minValueLocaiton):
    lowerIndex = minValueLocaiton[0]
    upperIndex = minValueLocaiton[1]
    colNames = distanceMatrix.columns
    rowcolString1 = colNames[lowerIndex]
    rowcolString2 = colNames[upperIndex]
    colName1 = colNames[lowerIndex].replace('S','')
    colName2 = colNames[upperIndex].replace('S','')
    newRowColName = f"S{colName1}{colName2}"
    
    # Remove columns
    distanceMatrix.drop(rowcolString1, axis=1, inplace=True)
    distanceMatrix.drop(rowcolString2, axis=1, inplace=True)
    # Remove rows
    distanceMatrix.drop(rowcolString1, inplace=True)
    distanceMatrix.drop(rowcolString2, inplace=True)


    # Filling row and col and index 0,0 with -9999
    rowCombo =distanceMatrix.iloc[0]
    newDF = pd.DataFrame(rowCombo, index=[0])
    distanceMatrix = pd.concat([newDF, distanceMatrix])
    rowArray  = np.zeros(distanceMatrix.shape[0])
    rowArray[:] = -9999
    distanceMatrix.insert(0, newRowColName, rowArray)
    distanceMatrix.iloc[0] = -9999
    distanceMatrix.index = distanceMatrix.columns
    
    # Setting diagonal = 0
    array = distanceMatrix.values
    np.fill_diagonal(array, 0)
    distanceMatrix_modified = pd.DataFrame(array, columns=distanceMatrix.columns, index=distanceMatrix.index)
    return distanceMatrix_modified

In [6]:
# From XSLX file read specific sheet name.
df = pd.read_excel("Hw1-words.xlsx", sheet_name='Proto-words')
df

,father,mother,one,eight,knee,nose,tongue,dog,egg,smoke,snow,to burn,to bite,to sing,to spit,dry
0,*ićä,*ava,*ikte,*kakteksan,*polwe,*nere,*kele,*pene,*muna,*sawe,*lume,*korpe,*pure,*mura,*śil'ke,*kuśka
1,*ićä,*emä,*ikte,*kakteksan,*polwe,*nenä,*kele,*pene,*muna,*sawe,*lume,*korpe,*pure,*laulu,*śil'ke,*kujwa
2,*ićä,*emä,*egy,*ńalɜ,*térd,*orr,*nyelv,*ämpɜ,*tojás,*pičɜ,*hó,*pilɜ,*harap,*énekel,*köp,*ϑasɜ
3,*ićä,*emä,*ikte,*ńalɜ,*sāns,*nere,*kele,*ämpɜ,*muna,*pičɜ,*tūjt,*pilɜ,*pure,*mura,*śil'ke,*ϑasɜ
4,*ićä,*ava,*ikte,*kakteksan,*polwe,*nere,*jylme,*pene,*muna,*šikš,*lume,*äŋɜ,*pure,*mura,*śil'ke,*kuśka
5,*ićä,*emä,*ikte,*kakteksan,*polwe,*nere,*kele,*pene,*muna,*sawe,*lume,*korpe,*pure,*laulu,*śil'ke,*kujwa


In [7]:
maxRowsCount = df.shape[0]
maxRowsCount
keys = (list('ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz'))[:maxRowsCount]
keys

['A', 'B', 'C', 'D', 'E', 'F']

In [8]:
dfKeyed = df.copy()
for col in df.columns:
    selectedCol = df[col]
    unique_values = selectedCol.unique()
    word2key = dict(zip(unique_values, keys))
    dfKeyed[col].replace(word2key, inplace=True)

In [9]:
dfKeyed

,father,mother,one,eight,knee,nose,tongue,dog,egg,smoke,snow,to burn,to bite,to sing,to spit,dry
0,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A
1,A,B,A,A,A,B,A,A,A,A,A,A,A,B,A,B
2,A,B,B,B,B,C,B,B,B,B,B,B,B,C,B,C
3,A,B,A,B,C,A,A,B,A,B,C,B,A,A,A,C
4,A,A,A,A,A,A,C,A,A,C,A,C,A,A,A,A
5,A,B,A,A,A,A,A,A,A,A,A,A,A,B,A,B


In [10]:
matrix_size = dfKeyed.shape[0]
NameIndexed = []
for i in range(matrix_size):
    NameIndexed.append("S"+str(i+1))
print("Column and Row names",NameIndexed)

Column and Row names ['S1', 'S2', 'S3', 'S4', 'S5', 'S6']


### Setting row/col names

In [11]:
distance_matrix = pd.DataFrame(index=range(df.shape[0]), columns=range(df.shape[0]))
distance_matrix = pd.DataFrame(distance_matrix, index=NameIndexed)
distance_matrix.columns = NameIndexed

In [12]:
min_value_location = []
min_value = 999999
for i in range (matrix_size):
    for j in range (matrix_size):
        col=j
        row =i
        row1 = df.iloc[row]
        row2 = df.iloc[col]
        distance = (row1 != row2).sum()
        if min_value>distance and distance!=0:
            min_value = distance
            min_value_location = (row,col)        
        distance_matrix.iloc[row,col]=distance

### <u>Distance matrix from Proto words</u>
**Initial distance matrix**

In [13]:
distance_matrix

,S1,S2,S3,S4,S5,S6
S1,0,4,15,8,3,3
S2,4,0,14,9,7,1
S3,15,14,0,9,15,14
S4,8,9,9,0,9,8
S5,3,7,15,9,0,6
S6,3,1,14,8,6,0


### Renaming columns as S1..Sn & finding the row/col index of the minimum distance

In [14]:
distance_matrix.columns = NameIndexed
distance_matrix.shape
lowerIndex, upperIndex = min_value_location
print(f"Minimum distance row/col index==> {min_value_location}")
print("Row==>", distance_matrix.columns[min_value_location[0]], "Col==>",distance_matrix.columns[min_value_location[1]] )

Minimum distance row/col index==> (1, 5)
Row==> S2 Col==> S6


In [15]:
newRowColName = f"S{lowerIndex+1}{upperIndex+1}"

### Creating a copy of distance matrix for calculation of 2nd step and removing the rows and column associated with minimum row/col distance.

In [16]:
distance_matrix_l1 = distance_matrix.copy()
distance_matrix_l1.drop(distance_matrix_l1.columns[upperIndex], axis=1, inplace=True)
distance_matrix_l1.drop(distance_matrix_l1.columns[lowerIndex], axis=1, inplace=True)

distance_matrix_l1.drop(distance_matrix_l1.index[upperIndex], inplace=True)
distance_matrix_l1.drop(distance_matrix_l1.index[lowerIndex], inplace=True)
print("Distance matrix after removing min value rows and columns")
distance_matrix_l1

Distance matrix after removing min value rows and columns


,S1,S3,S4,S5
S1,0,15,8,3
S3,15,0,9,15
S4,8,9,0,9
S5,3,15,9,0


### Adding row and col at index 0,0 with with the row/col name.

In [17]:
rowCombo =distance_matrix_l1.iloc[0]
newDF = pd.DataFrame(rowCombo, index=[0])
distance_matrix_l1 = pd.concat([newDF, distance_matrix_l1])
# rowCombo  = distance_matrix_l1[0]
rowArray  = np.zeros(distance_matrix_l1.shape[0])
rowArray[:] = np.nan
distance_matrix_l1.insert(0, f"S{lowerIndex+1}{upperIndex+1}", rowArray)

In [18]:
indexNames = distance_matrix_l1.index.tolist()
indexNames[0]= f"S{lowerIndex+1}{upperIndex+1}"
distance_matrix_l1.index= indexNames

#### Constructed a matrix with new colum/row name corresponding to min distance combo
e.g. If min distance value is at S3 & S5; new col name will be S35

In [19]:
distance_matrix_l1

,S26,S1,S3,S4,S5
S26,NaN,NaN,NaN,NaN,NaN
S1,NaN,0,15,8,3
S3,NaN,15,0,9,15
S4,NaN,8,9,0,9
S5,NaN,3,15,9,0


In [20]:
distance_matrix_l1_init = updateDistances(distance_matrix_l1, min_value_location)
distance_matrix_l1_init

S2_S1(4)   S6_S1(3)  
S2_S3(14)  S6_S3(14) 
S2_S4(9)   S6_S4(8)  
S2_S5(7)   S6_S5(6)  


,S26,S1,S3,S4,S5
S26,NaN,3.5,14.0,8.5,6.5
S1,3.5,0,15,8,3
S3,14.0,15,0,9,15
S4,8.5,8,9,0,9
S5,6.5,3,15,9,0


In [21]:
matrix2RowColMin = getMinLocation(distance_matrix_l1_init)

Min Location==> <bound method Location.text of <__main__.Location object at 0x00000297121E6310>> Min value==> 3
Row==> S1 Col==> S5


### Creating matrix and filling row=0; col=0 with dummy values -9999

In [22]:
updatedMatrix = removeMinDistanceRowCol(distance_matrix_l1_init, matrix2RowColMin)
updatedMatrix

TypeError: 'Location' object is not subscriptable

In [ ]:
updatedMatrix2 = updateDistances(updatedMatrix, matrix2RowColMin)
updatedMatrix2

In [ ]:
distance_matrix_l1_init